In [13]:
#coding=utf-8
import gdal
import matplotlib.pyplot as plt
import numpy as np  
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree, svm, naive_bayes,neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier,  GradientBoostingClassifier
from keras import models
import cv2
import os
from skimage import io as sio
from skimage.segmentation import felzenszwalb
from mylib import *

In [18]:
def felzenszwalb_test(img,sigma,kernel,k, min_size):
    # 先使用纹理特征滤波，再计算距离
    img = maxminNorm(img)

    # rescale scale to behave like in reference implementation
    k = float(k) / 255.
    img = cv2.GaussianBlur(img, (kernel, kernel), sigma)
    height, width = img.shape[:2]
    num = height * width
    edges = np.zeros(((height - 1) * width * 2 + height * (width - 1) * 2, 3))

    # 使用RGB距离，计算四邻域
    index = np.array([i for i in range(height * width)])
    index = index.reshape((height, width))
    to_left = np.sqrt(((img[:, 1:] - img[:, :-1]) ** 2).sum(axis=2))
    to_right = to_left
    to_up = np.sqrt(((img[1:] - img[:-1]) ** 2).sum(axis=2))
    to_down = to_up


    last, cur = 0, 0
    last, cur = cur, cur + (width - 1) * height
    edges[last: cur, 0] = index[:, 1:].reshape(-1)
    edges[last: cur, 1] = index[:, :-1].reshape(-1)
    edges[last: cur, 2] = to_left.reshape(-1)

    last, cur = cur, cur + (width - 1) * height
    edges[last: cur, 0] = index[:, :-1].reshape(-1)
    edges[last: cur, 1] = index[:, 1:].reshape(-1)
    edges[last: cur, 2] = to_right.reshape(-1)

    last, cur = cur, cur + (height - 1) * width
    edges[last: cur, 0] = index[1:].reshape(-1)
    edges[last: cur, 1] = index[:-1].reshape(-1)
    edges[last: cur, 2] = to_up.reshape(-1)

    last, cur = cur, cur + (height - 1) * width
    edges[last: cur, 0] = index[:-1].reshape(-1)
    edges[last: cur, 1] = index[1:].reshape(-1)
    edges[last: cur, 2] = to_down.reshape(-1)

    # 将边按照不相似度从小到大排序
    edges = [edges[i] for i in range(edges.shape[0])]
    edges.sort(key=lambda x: x[2])

    # 构建无向图（树）
    class universe(object):
        def __init__(self, n, k):
            self.f = np.array([i for i in range(n)])  # 树
            self.r = np.zeros_like(self.f)   # root
            self.s = np.ones((n))  # 存储像素点的个数
            self.t = np.ones((n)) * k  # 存储不相似度
            self.k = k

        def find(self, x):    # Find root of node x
            if x == self.f[x]:
                return x
            return self.find(self.f[x])

        def join(self, a, b):  # Join two trees containing nodes n and m
            if self.r[a] > self.r[b]:
                self.f[b] = a
                self.s[a] += self.s[b]
            else:
                self.f[a] = b
                self.s[b] += self.s[a]
                if self.r[a] == self.r[b]:
                    self.r[b] += 1

    u = universe(num, k)
    for edge in edges:
        a, b = u.find(int(edge[0])), u.find(int(edge[1]))
        if ((a != b) and (edge[2] <= min(u.t[a], u.t[b]))):
            # 更新类标号：将的类a,b标号统一为的标号a。更新该类的不相似度阈值为：k / (u.s[a]+u.s[b])
            u.join(a, b)
            a = u.find(a)
            u.t[a] = edge[2] + k / u.s[a]

    for edge in edges:
        a, b = u.find(int(edge[0])), u.find(int(edge[1]))
        if ((a != b) and ((u.s[a] < min_size) or u.s[b] < min_size)):
            # 分割后会有很多小区域，当区域像素点的个数小于min_size时，选择与其差异最小的区域合并
            u.join(a, b)

    dst = np.zeros_like((img.shape[0],img.shape[1],3))

    def locate(index):
        return index // width, index % width

    avg_color = np.zeros((num, 3))

    for i in range(num):
        f = u.find(i)
        x, y = locate(i)
        avg_color[f, :] += img[x, y, :3] / u.s[f]

    for i in range(height):
        for j in range(width):
            f = u.find(i * width + j)
            dst[i, j, :] = avg_color[f, :]
    return dst


# if __name__ == '__main__':
#     sigma = 0.5
#     kernel = 3
#     K, min_size = 500, 50
#     image = sio.imread(r"E:/project/codes/Unsupervised-Segmentation/image/15_12_11.tif")
#     # skimage自带的felzenszwalb算法
#     seg1 = felzenszwalb(image, scale=K, sigma=sigma, min_size=min_size)
#     # felzenszwalb算法的实现，相比于上一种，区别主要在四邻域和颜色还原
#     seg3=felzenszwalb_test(image, sigma, kernel,K, min_size)

#     fig = plt.figure(figsize=(24,8))
#     a = fig.add_subplot(131)
#     plt.imshow(image)
#     a.set_title("image")

#     a = fig.add_subplot(132)
#     plt.imshow(seg1)
#     a.set_title("seg1")

#     a = fig.add_subplot(133)
#     plt.imshow(seg3)
#     a.set_title("seg3")
#     plt.show()

In [5]:
image = sio.imread(r"E:/project/codes/Unsupervised-Segmentation/image/15_12_11.tif")

In [6]:
image = cv2.GaussianBlur(image, (3, 3), 0.5)

In [9]:
image[1:].shape

(255, 256, 3)

In [10]:
# prepare data
path=r"E:\project\images\researchImage\HR\originData"
img1=getBands(os.path.join(path,"000326.tif"))
img2=getBands(os.path.join(path,"040121.tif"))
data1=img1[600:1000,600:1000,:]
data2=img2[600:1000,600:1000,:]
indexName=["NDVI","NDWI","MSAVI","MTVI","VARI"]
labdata1=returnLabeldata(data1)
labdata2=returnLabeldata(data2)
data1norm=maxminNorm(data1)
data2norm=maxminNorm(data2)
labdata1norm=maxminNorm(labdata1)
labdata2norm=maxminNorm(labdata2)
data1total=np.concatenate((data1norm,labdata1norm),axis=2)
data2total=np.concatenate((data2norm,labdata2norm),axis=2)

In [19]:
seg3=felzenszwalb_test(data1total, sigma, kernel,K, min_size)
plt.imshow(seg3)

IndexError: too many indices for array